In [5]:
import pandas as pd
import numpy as np

In [6]:
# Load the google form and venmo csv files
google_form = 'all_tickets.csv'
venmo_form = 'all_venmo.csv'
name = '3_9'
google_df = pd.read_csv(google_form)
venmo_df = pd.read_csv(venmo_form)
venmo_issues = pd.read_csv(venmo_form)
index_list = np.arange(0, len(venmo_df))
venmo_df['index'] = index_list
venmo_issues['index'] = index_list


In [7]:
# Checks if there exists a venmo payment with the given Venmo Username
# If there are multiple payments / people being paid for by the same Venmo Username, classify it as an issue to review
venmo_verified_list = []
venmo_verified_index = []
def verify(username):
    # Check if multiple google form submissions have the same venmo username
    google_rows = google_df[google_df['Input Venmo name associated with the Venmo handle.'].str.lower().str.strip() == username]
    if (len(google_rows) != 1):
        return False
    # Check if multiple venmo payments have the same venmo username
    venmo_rows = venmo_df[venmo_df['Venmo Username'].str.lower() == username]
    if (len(venmo_rows) != 1):
        return False
    # Check that amount paid is not greater than $10, but also not less than $9.70
    str_amount = venmo_rows.iloc[0]['Amount']
    num_amount = float(str_amount.replace("$", ""))
    if num_amount > 10 or num_amount < 9.7:
        return False
    venmo_verified_list.append(venmo_rows.iloc[0])
    venmo_verified_index.append(venmo_rows.iloc[0]['index'])
    # Otherwise, verify
    return True

In [8]:
cash_list = []
issues_list = []
verified_list = []

# Key words for cash payments (can add more if students come up with brand new bright-minded ways to mess this up)
# cash_strings = ["cash", "n/a"]

# Checking every google form submission
for index, ticket in google_df.iterrows():
    first_name = str(ticket['First Name']).lower().strip()
    last_name = str(ticket['Last Name']).lower().strip()
    automated_email = str(ticket['Email Address']).lower().strip()
    dup_rows = google_df[google_df['First Name'].str.lower().str.strip() == first_name]
    dup_rows = dup_rows[dup_rows['Last Name'].str.lower().str.strip() == last_name]
    dup_rows = dup_rows[dup_rows['Email Address'].str.lower().str.strip() == automated_email]
    if len(dup_rows) > 1:
        issues_list.append(ticket)
        continue

    venmo_name = str(ticket['Input Venmo name associated with the Venmo handle.']).lower().strip()
    # venmo_handle = str(ticket['Input Venmo handle that completed your transaction [enter "cash" if paying by cash at Sproul]']).lower()
    payment_type = str(ticket['Did you Venmo @isa-berkeley the necessary amount? [$10]'])

    # Cash check (if any string in the cash_strings list is present in the Venmo Username or Handle)
    # if any(cash in venmo_name for cash in cash_strings) or any(cash in venmo_handle for cash in cash_strings):
    if payment_type == "Alternate Payment":
        cash_list.append(ticket)
    # Verify Check
    elif verify(venmo_name):
        verified_list.append(ticket)
    # For the monkeys and dum dums who can't fill out a form right
    else:
        issues_list.append(ticket)

for i in venmo_verified_index:
    venmo_issues = venmo_issues[venmo_issues['index'] != i]

verified_file, cash_file, issues_file, venmo_file, venmo_issues_file = 'verified_all.csv', 'cash_all.csv', 'issues_all.csv', 'verified_venmo_all.csv', 'venmo_issues_all.csv'
verified, cash, issues, verified_venmo = pd.DataFrame(verified_list), pd.DataFrame(cash_list), pd.DataFrame(issues_list), pd.DataFrame(venmo_verified_list)

verified_venmo = verified_venmo.drop(columns=['index'])
venmo_issues = venmo_issues.drop(columns=['index'])

verified.to_csv(verified_file, index = False)
cash.to_csv(cash_file, index = False)
issues.to_csv(issues_file, index = False)
verified_venmo.to_csv(venmo_file, index = False)
venmo_issues.to_csv(venmo_issues_file, index = False)


In [11]:
print(len(verified))
print(len(verified_venmo))
print(len(cash))
print(len(issues))
print(len(venmo_issues))

864
864
191
445
502
